# Python Tutorial++

Addendum to [Python 3.5 Tutorial](https://drive.google.com/open?id=1B8IvfcO7Gt_lXT6a-Bbvpqe4BXi5SW4U)

## Match 💼

In [ ]:
# 1. Take an expression and compare its value to successive patterns
def http_error(status):
  match status:
    case 400:
      return "Bad request"
    case 404:
      return "Not found"
    case 401 | 403:       # multiple literals
      return "Not allowed"
    case _:               # wildcard
      return "Something's wrong with the Internet"

# 2. Bind variables
def point_name(point):
  match point:
    case (0, 0):
      return "Origin"
    case (0, y):
      return f"Y={y}"
    case (x, 0):
      return f"X={x}"
    case (x, y) if x == y:  # guard match
      return f"Y=X at {x}"
    case (x, y):
      return f"X={x}, Y={y}"
    case _:
      raise ValueError("Not a point")


## Enum ⛳
Set of symbolic names bound to unique values

In [ ]:
from enum import Enum, Flag, auto

class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

class Days(Flag):
    MONDAY = 1
    TUESDAY = 2
    WEDNESDAY = 4
    THURSDAY = 8
    FRIDAY = 16
    SATURDAY = 32
    SUNDAY = 64
    WEEKEND = SATURDAY | SUNDAY

class AutoDays(Flag):
    MONDAY = auto()
    TUESDAY = auto()
    WEDNESDAY = auto()
    THURSDAY = auto()
    FRIDAY = auto()
    SATURDAY = auto()
    SUNDAY = auto()
    WEEKEND = SATURDAY | SUNDAY

for day in AutoDays.WEEKEND:
    print(type(day), day, day.name, day.value)

<flag 'AutoDays'> AutoDays.SATURDAY SATURDAY 32
<flag 'AutoDays'> AutoDays.SUNDAY SUNDAY 64


## Functional Programming

In [ ]:
# 1. Decorators
def print_args(fn):
  def decorated_fn(*args, **kwargs):
    print(f'Positional args: {args}')
    print(f'Keyword args: {kwargs}')
    return fn(*args, **kwargs)
  return decorated_fn

@print_args  # Equivalent to add = print_args(add)
def add(a, b):
  return a + b

print(add(2, 3))

# 2. Helper Generators: map, filter

numbers= [1, 2, 3, 4, 5]
print(*map(lambda x: x * x, numbers))
print(*filter(lambda x: x % 2 == 0, numbers))

# 3. Functools

from functools import cache, partial, reduce

@cache
def factorial(n):
  return n * factorial(n - 1) if n else 1
print(factorial(5), factorial(6))

relu = partial(max, 0)
print(relu(-1), relu(1))

def factoriall(n):
  return reduce(lambda x, y: x * y, range(1, n + 1))
print(factoriall(5))

Positional args: (2, 3)
Keyword args: {}
5
1 4 9 16 25
2 4
120 720
0 1
120


## Libraries

In [ ]:
# 1. collections

from collections import Counter, defaultdict, deque, namedtuple, OrderedDict

letters = Counter("mississippi")
print(*letters, letters.total())
print(*letters.elements())
print(letters.most_common(2))

queue = deque([1, 2, 3])
queue.appendleft(0)
print(queue.popleft())

Point = namedtuple('Point', ['x', 'y'])
p = Point(1, 1)
print(f"{p.x=}, {p.y=}")



# 2. itertools


# 3. pickle: serialize/unserialize python objects in binary



m i s p 11
m i i i i s s s s p p
[('i', 4), ('s', 4)]
p.x=1, p.y=1
0


## Asyncio

## Miscellaneous

In [95]:
# 1. Walrus Operator: assign values to variables as part of a larger expression:
suffix = []
word = list("prefix_suffix")
while (c := word.pop()) != "_":
  suffix.append(c)
print("".join(word), "".join(reversed(suffix)))


# 2. F-strings: include the value of an expression inside a string using {}
# Optionally use '=' to include the expression text
# Optionally use ':' to provide a format specifier
two_thirds = 2/3
print(f"Super majority requires {two_thirds=:.2%}")


# 3. Super: Call methods of the parent class:
class A:
  def __init__(self):
    print("A")

class B(A):
  def __init__(self):
    print("B")

class C(B):
  def __init__(self):
    super().__init__()  # Same as super(C, self).__init__()
    print("C")

c = C()


# 4. Context Manager:
# . Call expression to obtain a context manager
# . Store the context manager’s .__enter__() and .__exit__() methods
# . Call .__enter__() and bind its return value to the optional target variable
# . Execute the with code block
# . Call .__exit__() when the with code block finishes
import os
with os.scandir(".") as entries:
  for entry in entries:
    print(entry.name, "->", entry.stat().st_size, "bytes")

prefix suffix
Super majority requires two_thirds=66.67%
B
C
.config -> 4096 bytes
sample_data -> 4096 bytes
